In [2]:
# -*- coding: utf-8 -*-
import pandas as pd
import datetime
from tqdm import *
import numpy as np
from scipy import stats
df=pd.read_csv(r'C:\luyan\paper_code\data\hourly.csv')
df.head(5)

,day_id,id,temp_in,con
0,2016070107,0007A88A4D8D,24.7,0
1,2016070108,0007A88A4D8D,25.4,0
2,2016070109,0007A88A4D8D,25.5,0
3,2016070110,0007A88A4D8D,25.0,0
4,2016070111,0007A88A4D8D,26.0,0


In [3]:
listid=[]
list_trigger_temp=[]


for nameid,dfi in tqdm(df.groupby('id')):
    dfi=dfi.reset_index(drop=True)
    for i in range(1,len(dfi)):
        a=dfi.loc[i,'con']
        b=dfi.loc[i-1,'con']
        time0=dfi.loc[i-1,'day_id']
        time1=dfi.loc[i,'day_id']
        time0_format=datetime.datetime.strptime(str(time0),'%Y%m%d%H')
        time1_format=datetime.datetime.strptime(str(time1),'%Y%m%d%H')
        delta=time1_format-time0_format
        delta_hour=delta.seconds/3600
        # print (delta_hour)
        if b==0 and a!=0:            
            trigger_temp=dfi.loc[i-1,'temp_in']
            list_trigger_temp.append(trigger_temp)
            listid.append(nameid)
            


dftrigger_temp=pd.DataFrame()
dftrigger_temp['id']=listid
dftrigger_temp['trigger_temp']=list_trigger_temp

def droplevel(df):
    df.columns=df.columns.droplevel(0)

def aggfeature(df,featurename):
    df_mean=pd.pivot_table(df,values=featurename,index='id',aggfunc=[np.mean])
    droplevel(df_mean)
    df_mean.reset_index(inplace=True)
    df_mean.columns=['id',featurename+'_mean']
    
    df_median=pd.pivot_table(df,values=featurename,index='id',aggfunc=[np.median])
    droplevel(df_median)
    df_median.reset_index(inplace=True)
    df_median.columns=['id',featurename+'_median']
    
    
    df_std=pd.pivot_table(df,values=featurename,index='id',aggfunc=[np.std])
    droplevel(df_std)
    df_std.reset_index(inplace=True)
    df_std.columns=['id',featurename+'_std']
    
    
    
    df_skew=pd.pivot_table(df,values=featurename,index='id',aggfunc=[stats.skew])
    droplevel(df_skew)
    df_skew.reset_index(inplace=True)
    df_skew.columns=['id',featurename+'_skew']
    
    df_kurt=pd.pivot_table(df,values=featurename,index='id',aggfunc=[stats.kurtosis])
    droplevel(df_kurt)
    df_kurt.reset_index(inplace=True)
    df_kurt.columns=['id',featurename+'_kurt']
    
    df_merge=pd.merge(df_mean,df_median,on='id')
    df_merge=pd.merge(df_merge,df_std,on='id')
    df_merge=pd.merge(df_merge,df_skew,on='id')
    df_merge=pd.merge(df_merge,df_kurt,on='id')
    
    return df_merge






df_trigger_temp_features=aggfeature(dftrigger_temp,'trigger_temp')


#df_trigger_temp_features.to_csv(r'C:\luyan\paper_code\data\feature_data\trigger_temp.csv',index=False)
df_trigger_temp_features.head(5)

100%|██████████████████████████████████████████████████████████████████████████████| 1289/1289 [00:50<00:00, 25.45it/s]


,id,trigger_temp_mean,trigger_temp_median,trigger_temp_std,trigger_temp_skew,trigger_temp_kurt
0,0007A88A4D8D,27.842029,27.80,1.502782,-0.076451,0.755470
1,0007A88B9A54,26.964615,27.10,1.139686,-0.498886,0.186714
2,0007A88B9C97,28.590000,28.75,2.076091,0.267460,-0.598157
3,0007A88BA9B4,31.655556,31.55,0.931932,0.856593,0.032713
4,0007A88BCE80,30.160465,29.90,1.448010,0.265062,-0.487133
